In [5]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm
import numpy as np

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import root_mean_squared_error

In [6]:
train_df = pd.read_csv("data/train.csv", index_col="id")
test_df = pd.read_csv("data/test.csv", index_col="id")

train_extra_df = pd.read_csv("data/training_extra.csv", index_col="id")

In [7]:
def convert_cols(df, cat_feats):
    for feat in cat_feats:
        df[feat] = df[feat].astype("category")


    return df

CONT_FEATS = ["Compartments", "Weight Capacity (kg)"]
CAT_FEATS = [name for name in train_df.columns.to_list() if name not in CONT_FEATS and name != "Price"]
TARGET_FEAT = "Price"

train_df = convert_cols(train_df, CAT_FEATS)
test_df = convert_cols(test_df, CAT_FEATS)
train_extra_df = convert_cols(train_extra_df, CAT_FEATS)

In [8]:
tr_dummy_df = pd.get_dummies(train_df, drop_first=True)
te_dummy_df = pd.get_dummies(test_df, drop_first=True)
tr_exdummy_df = pd.get_dummies(train_extra_df, drop_first=True)

In [9]:
tr_x, val_x, tr_y, val_y= train_test_split(tr_dummy_df.drop(["Price"], axis=1), tr_dummy_df["Price"], test_size=0.2, random_state=0)

In [14]:
tr_x

,Compartments,Weight Capacity (kg),Brand_Jansport,Brand_Nike,Brand_Puma,Brand_Under Armour,Material_Leather,Material_Nylon,Material_Polyester,Size_Medium,Size_Small,Laptop Compartment_Yes,Waterproof_Yes,Style_Messenger,Style_Tote,Color_Blue,Color_Gray,Color_Green,Color_Pink,Color_Red
id,,,,,,,,,,,,,,,,,,,,
154452,8.0,16.817078,False,False,False,True,False,False,True,False,True,False,True,False,False,False,False,False,False,True
297220,10.0,16.750179,False,False,False,False,True,False,False,True,False,False,True,False,False,True,False,False,False,False
177,8.0,13.262331,False,False,False,True,False,True,False,True,False,True,True,True,False,False,False,True,False,False
44603,10.0,29.694404,False,False,False,False,False,False,True,False,True,False,False,False,True,False,True,False,False,False
158177,2.0,11.291522,False,False,True,False,False,True,False,False,True,False,True,True,False,True,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
211543,10.0,5.000000,False,False,False,True,True,False,False,False,False,True,False,False,False,False,True,False,False,False
86293,1.0,26.255908,False,False,True,False,False,False,False,False,True,False,True,True,False,False,False,False,False,True
122579,9.0,24.351598,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,True,False,False


In [ ]:
def eval_metric(preds):
    rmse = root_mean_squared_error(val_y, preds)
    print(f"RMSE: {rmse}")

def objective(trial):
    n_estimators = trial.suggest_int('n_estimators', 100, 1000)
    max_depth = trial.suggest_int('max_depth', 10, 50)
    min_samples_split = trial.suggest_int('min_samples_split', 2, 32)
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 32)
    max_features = trial.suggest_categorical('max_features', ['sqrt', 'log2'])


rf = RandomForestRegressor(random_state=0)
